In [1]:
import numpy as np
import sys
from MosaicAnalysis import Analysis, MatrixSpectrum
from MosaicsCreation import Mosaics
from data import KyotoNaturalImages
import torch
import matplotlib

/home/david/anaconda3/envs/reteffcod/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400430266/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
save_name = 'Artificial1'
kernel_size = 18
images = KyotoNaturalImages('kyoto_natim', kernel_size, circle_masking = True, device = "cuda:0", n_colors = 2, normalize_color = True, restriction = 'True', remove_mean = False)
images.covariance()

root kyoto_natim
Loading 62 images from kyoto_natim ...


Taking samples for covariance calculation: 100%|████████| 100000/100000 [00:01<00:00, 53050.34it/s]


tensor([[9.9291e-09, 8.9754e-09, 7.7755e-09,  ..., 3.1654e-09, 3.1421e-09,
         3.1087e-09],
        [8.9754e-09, 9.9108e-09, 8.9693e-09,  ..., 3.1934e-09, 3.1700e-09,
         3.1423e-09],
        [7.7755e-09, 8.9693e-09, 9.9303e-09,  ..., 3.2351e-09, 3.2142e-09,
         3.1857e-09],
        ...,
        [3.1654e-09, 3.1934e-09, 3.2351e-09,  ..., 9.8626e-09, 8.9909e-09,
         7.8001e-09],
        [3.1421e-09, 3.1700e-09, 3.2142e-09,  ..., 8.9909e-09, 9.8735e-09,
         9.0072e-09],
        [3.1087e-09, 3.1423e-09, 3.1857e-09,  ..., 7.8001e-09, 9.0072e-09,
         9.8704e-09]], device='cuda:0')

In [ ]:
input_noise = 30
output_noise = 0.1

gain = 10
bias = 1.6

neurons_per_mosaic = np.array([50,50,50,50]) #All numbers must be even
a = 4; b = 0; c = 0.99

#a = np.log(a)
#b = 1/(1/np.log(b) - 1/a)
#c = np.log(c/(1-c))

#params = np.array([[a,b,c, -0.5,    a,b,c,-0.5],
#                   [a,b,c, 0.5,    a,b,c,0.5]]).T
#params = np.array([[-0.4, -0.6,  0.2,  2, -0.3, -0.3, -1,  1.6], #These are original values from a trained model
#                   [-0.4, -0.6,  0.2,  -2, -0.3, -0.3, -1,  -1.6]]).T

params = np.array([[-0.4, -0.6,  0.2,  2, -0.3, -0.3, -1,  1.6], #These are original values from a trained model
                   [-0.4, -0.6,  0.2,  -2, -0.3, -0.3, -1,  -1.6],
                   [-0.4, -0.6,  0.2,  2, -0.3, -0.3, -1,  -1.6],
                   [-0.4, -0.6,  0.2,  -1.5, -0.3, -0.3, -1,  1.6]]).T

mosaic = Mosaics(params, neurons_per_mosaic, kernel_size, gain, bias, input_noise, output_noise, images = images)
spec = MatrixSpectrum(mosaic.model, mosaic.images, batch_size = 128, input_noise = input_noise, output_noise = output_noise)
print(np.mean(spec.numerator) - np.mean(spec.denominator))

In [ ]:
mosaic.save(save_name)
n_clusters_global = 2
n_comps_global = 3 
rad_dist_global = 5 

test = Analysis('artificial_mosaics/' + save_name)
test(n_comps_global, rad_dist_global, n_clusters_global)
%matplotlib inline
test.plot_rad_avg(test.rad_avg[0,:,:])